In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
base_data_dir = '/srv/scratch/z5370003/projects/data/groundwater/FEFLOW/coastal/variable_density/'

bc_dir_name = 'bc_data'
filtered_dir_name = 'filter_all_ts'
bc_out_dir_name = 'bc_data_corrected'

In [3]:
filtered_data_path = os.path.join(base_data_dir, filtered_dir_name)
bc_data_path = os.path.join(base_data_dir, bc_dir_name)
bc_out_data_path = os.path.join(base_data_dir, bc_out_dir_name)
os.makedirs(bc_out_data_path, exist_ok=True)

In [4]:
for ts_file in tqdm(os.listdir(filtered_data_path)):
    filtered_data_at_ts_path = os.path.join(filtered_data_path, ts_file)
    bc_data_at_ts_path = os.path.join(bc_data_path, ts_file)

    drop_data_columns = ['head', 'pressure', 'mass_concentration']
    filtered_df = pd.read_csv(filtered_data_at_ts_path).drop(columns=drop_data_columns)

    bc_cols = ['node', 'mass_concentration_bc', 'hydraulic_head_bc']
    bc_df = pd.read_csv(bc_data_at_ts_path).loc[:, bc_cols]
    bc_df.rename(columns={'hydraulic_head_bc':'head_bc'}, inplace=True)

    merged_df = pd.merge(filtered_df, bc_df, on='node', how='left')
    break
    # merged_df.to_csv(os.path.join(bc_out_data_path, ts_file), index=False)
    

  0%|          | 0/1909 [00:00<?, ?it/s]


In [13]:
# Create color array: use head_bc where available, otherwise use a special value for grey
colors = merged_df['head_bc'].fillna(-999)  # Use -999 as placeholder for NaN

fig = go.Figure(data=[go.Scatter3d(
    x=merged_df['X'],
    y=merged_df['Y'],
    z=merged_df['Z'],
    mode='markers',
    marker=dict(
        size=5,
        color=colors,
        colorscale=[[0, 'grey'], [0.001, '#440154'], [0.1111111111111111, '#482878'], 
                    [0.2222222222222222, '#3e4989'], [0.3333333333333333, '#31688e'], 
                    [0.4444444444444444, '#26828e'], [0.5555555555555556, '#1f9e89'], 
                    [0.6666666666666666, '#35b779'], [0.7777777777777778, '#6ece58'], 
                    [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
        cmin=merged_df['head_bc'].min(),
        cmax=merged_df['head_bc'].max(),
        showscale=True,
        colorbar=dict(title="Hydraulic Head BC")
    ),
    text=merged_df['head_bc'].fillna('No BC'),
    hovertemplate='X: %{x}<br>Y: %{y}<br>Z: %{z}<br>Head BC: %{text}<extra></extra>'
)])

fig.update_layout(
    title='Hydraulic Head BC - 3D Scatter Plot',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()